In [1]:
!pip install diffusers==0.31.0 transformers accelerate safetensors torch --quiet
!pip install streamlit pillow tqdm realesrgan gdown --quiet
!pip install xformers --quiet --upgrade
#Install Dependencies (GPU/CPU Auto-Detect)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.4/299.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.

In [2]:
#Load Stable Diffusion Model
import torch
from diffusers import StableDiffusionPipeline

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    safety_checker=None  # custom safety implemented later
)

if device == "cuda":
    pipe.enable_xformers_memory_efficient_attention()
    pipe.to("cuda")
else:
    pipe.to("cpu")


Running on: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [3]:
#Create Safety Filter (NSFW Check)
from transformers import CLIPProcessor, CLIPModel
import torch.nn.functional as F

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def nsfw_filter(image):
    inputs = clip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = clip_model.get_image_features(**inputs)
    score = float(logits.sum())
    return score < 0  # threshold heuristic


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [4]:
#Generation Function with Metadata Saving
import time, os, json
from PIL import Image

def generate_and_save(prompt, negative_prompt, num_steps, guidance, width, height, seed=None):
    generator = torch.Generator(device=device)
    if seed is not None:
        generator.manual_seed(seed)

    output = pipe(prompt=prompt,
                  negative_prompt=negative_prompt,
                  num_inference_steps=num_steps,
                  guidance_scale=guidance,
                  width=width,
                  height=height,
                  generator=generator).images[0]

    # Safety Check
    if nsfw_filter(output):
        raise Exception("NSFW Content detected. Try different prompt.")

    ts = int(time.time())
    os.makedirs("generated/images", exist_ok=True)
    os.makedirs("generated/metadata", exist_ok=True)

    filename = f"generated/images/{ts}.png"
    output.save(filename)

    metadata = {
        "prompt": prompt,
        "negative_prompt": negative_prompt,
        "steps": num_steps,
        "guidance": guidance,
        "width": width,
        "height": height,
        "seed": seed,
        "timestamp": ts
    }
    with open(f"generated/metadata/{ts}.json", "w") as f:
        json.dump(metadata, f, indent=4)

    return filename


In [5]:
#Optional Upscaler (Real-ESRGAN)
def upscale_image(image_path):
    from subprocess import run
    upscaled_path = image_path.replace(".png", "_up.png")
    run(f"realesrgan-ncnn-vulkan -i {image_path} -o {upscaled_path}", shell=True)
    return upscaled_path


In [6]:
#Build Streamlit Web UI
%%writefile app.py
import streamlit as st
from PIL import Image
import os
from main import generate_and_save, upscale_image

st.title(" AI-Powered Text to Image Generator (Stable Diffusion) ")

prompt = st.text_input("Enter Prompt:", "A futuristic city at sunset, ultra realistic")
negative_prompt = st.text_input("Negative Prompt:", "blurry, lowres, watermark")
steps = st.slider("Inference Steps:", 10, 100, 40)
guidance = st.slider("Guidance Scale:", 1.0, 15.0, 7.5)
width = st.selectbox("Width:", [512, 768, 1024])
height = st.selectbox("Height:", [512, 768, 1024])

upscale = st.checkbox("Upscale Image (Optional)")

if st.button("Generate"):
    with st.spinner("Generating..."):
        try:
            img_path = generate_and_save(prompt, negative_prompt, steps, guidance, width, height)
            st.image(img_path, caption="Generated Image")

            if upscale:
                up_path = upscale_image(img_path)
                st.image(up_path, caption="Upscaled Image")

            st.success("Done!")

        except Exception as e:
            st.error(str(e))


Writing app.py


In [7]:
%%writefile main.py
from app import *


Writing main.py


In [8]:
!streamlit run app.py --server.port 6006 & npx localtunnel --port 6006


⠙⠹⠸⠼⠴⠦⠧⠇⠏

⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) 
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:6006
  Network URL: http://172.28.0.12:6006
  External URL: http://34.142.189.33:6006

  Stopping...
^C


In [9]:
from pyngrok import ngrok
ngrok.set_auth_token("360QQO3tjj9eASFk81wxtdrg7Uo_5K2VuEUMpALinnaYw7t74")
NGROK_AUTH_TOKEN = "360QQO3tjj9eASFk81wxtdrg7Uo_5K2VuEUMpALinnaYw7t74"



ModuleNotFoundError: No module named 'pyngrok'

In [10]:
import subprocess, threading

# This cell assumes app.py is already defined as a Streamlit application.
# If app.py is not yet created or correctly written, the Streamlit app will not start.
# The app.py content was last updated in cell `tKiChpnZ_TnP`.

def run():
    subprocess.run(["streamlit", "run", "app.py", "--server.port=8501", "--server.headless=True"])

# Run Streamlit in a separate thread to not block the Colab environment
threading.Thread(target=run).start()


In [11]:
!pip install pyngrok streamlit -q
from pyngrok import ngrok
import subprocess, threading

NGROK_AUTH_TOKEN = "360QQO3tjj9eASFk81wxtdrg7Uo_5K2VuEUMpALinnaYw7t74"  # Must start with 2

ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok.kill()

public_url = ngrok.connect(8501)
print("Public URL:", public_url)

with open("app.py", "w") as f:
    f.write("import streamlit as st\nst.title('Streamlit Works!')")

def run():
    subprocess.run(["streamlit", "run", "app.py", "--server.port=8501"])

threading.Thread(target=run).start()


Public URL: NgrokTunnel: "https://3550f275513f.ngrok-free.app" -> "http://localhost:8501"
